## Importa bibliotecas

In [1]:
import pandas as pd

## Carrega dados

In [11]:
filmes = pd.read_csv("movie_lens/movies.csv")

filmes.columns = ['filmeId', 'titulo', 'genero']
filmes.set_index = 'filmeId'
filmes.head()

,filmeId,titulo,genero
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
notas = pd.read_csv("movie_lens/ratings.csv")
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
notas.describe()

,usuarioId,filmeId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


## Primeira tentativa de recomendação

In [15]:
total_de_votos = notas.filmeId.value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [13]:
filmes.loc[356]

filmeId                             412
titulo     Age of Innocence, The (1993)
genero                            Drama
Name: 356, dtype: object

In [16]:
filmes['total_de_votos'] = total_de_votos
filmes.head()

,filmeId,titulo,genero,total_de_votos
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,215.0
2,3,Grumpier Old Men (1995),Comedy|Romance,110.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,52.0
4,5,Father of the Bride Part II (1995),Comedy,7.0


In [19]:
filmes.sort_values("total_de_votos", ascending=False).head(10)

,filmeId,titulo,genero,total_de_votos
356,412,"Age of Innocence, The (1993)",Drama,329.0
318,360,I Love Trouble (1994),Action|Comedy,317.0
296,338,Virtuosity (1995),Action|Sci-Fi|Thriller,307.0
593,735,Cemetery Man (Dellamorte Dellamore) (1994),Horror,279.0
2571,3439,Teenage Mutant Ninja Turtles II: The Secret of...,Action|Children|Fantasy,278.0
260,300,Quiz Show (1994),Drama,251.0
480,548,Terminal Velocity (1994),Action|Mystery|Thriller,238.0
110,128,Jupiter's Wife (1994),Documentary,237.0
589,726,Last Dance (1996),Drama,224.0
527,616,"Aristocats, The (1970)",Animation|Children,220.0


In [22]:
notas_medias = notas.groupby('filmeId').mean().nota
notas_medias.head()

filmeId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: nota, dtype: float64

## Uma segunda heurística

In [25]:
# Recomendar pelas notas medias mais altas
filmes['nota_media'] = notas_medias
filmes.sort_values("nota_media", ascending=False).head(10)

,filmeId,titulo,genero,total_de_votos,nota_media
6611,55805,Before the Devil Knows You're Dead (2007),Crime|Drama|Thriller,1.0,5.0
5328,8861,Resident Evil: Apocalypse (2004),Action|Horror|Sci-Fi|Thriller,1.0,5.0
8238,104243,Riddick (2013),Action|Sci-Fi|Thriller|IMAX,1.0,5.0
3096,4156,Company Man (2000),Comedy,1.0,5.0
3086,4143,Valentine (2001),Horror|Mystery,1.0,5.0
1140,1493,Love and Other Catastrophes (1996),Romance,1.0,5.0
1151,1514,Temptress Moon (Feng Yue) (1996),Romance,2.0,5.0
3073,4126,Less Than Zero (1987),Drama,1.0,5.0
4454,6577,Kickboxer 2: The Road Back (1991),Action|Drama,1.0,5.0
4495,6654,Atragon (Kaitei Gunkan) (1963),Adventure|Sci-Fi,1.0,5.0


In [27]:
# filmes com mais de 10 votos
filmes.query("total_de_votos >= 10").sort_values("nota_media", ascending=False).head(10)


,filmeId,titulo,genero,total_de_votos,nota_media
1041,1353,"Mirror Has Two Faces, The (1996)",Comedy|Drama|Romance,11.0,4.590909
3451,4708,Marat/Sade (1966),Drama|Musical,11.0,4.545455
1178,1569,My Best Friend's Wedding (1997),Comedy|Romance,12.0,4.541667
1104,1437,"Cement Garden, The (1993)",Drama,20.0,4.475000
2360,3127,Holy Smoke (1999),Comedy|Drama,12.0,4.458333
1217,1616,"Peacemaker, The (1997)",Action|Thriller|War,15.0,4.433333
318,360,I Love Trouble (1994),Action|Comedy,317.0,4.429022
951,1252,Chinatown (1974),Crime|Film-Noir|Mystery|Thriller,14.0,4.392857
1927,2555,Baby Geniuses (1999),Comedy,10.0,4.350000
3468,4732,Bubble Boy (2001),Comedy,18.0,4.333333


In [28]:
# filmes com mais de 50 votos
filmes.query("total_de_votos >= 50").sort_values("nota_media", ascending=False).head(10)


,filmeId,titulo,genero,total_de_votos,nota_media
318,360,I Love Trouble (1994),Action|Comedy,317.0,4.429022
858,1129,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,192.0,4.289062
2959,3967,Billy Elliot (2000),Drama,218.0,4.272936
1276,1693,Amistad (1997),Drama|Mystery,57.0,4.271930
750,984,"Pompatus of Love, The (1996)",Comedy|Drama,97.0,4.268041
904,1202,Withnail & I (1987),Comedy,84.0,4.261905
1221,1621,Soul Food (1997),Drama,129.0,4.259690
1213,1612,"Kiss Me, Guido (1997)",Comedy,126.0,4.250000
912,1211,"Wings of Desire (Himmel über Berlin, Der) (1987)",Drama|Fantasy|Romance,100.0,4.240000
50,55,Georgia (1995),Drama,204.0,4.237745
